In [1]:
!pip3 install openai


  Using cached openai-1.52.0-py3-none-any.whl.metadata (24 kB)
  Using cached anyio-4.6.2.post1-py3-none-any.whl.metadata (4.7 kB)
  Using cached distro-1.9.0-py3-none-any.whl.metadata (6.8 kB)
  Using cached httpx-0.27.2-py3-none-any.whl.metadata (7.1 kB)
  Using cached jiter-0.6.1-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.2 kB)
  Using cached pydantic-2.9.2-py3-none-any.whl.metadata (149 kB)
  Using cached sniffio-1.3.1-py3-none-any.whl.metadata (3.9 kB)
  Using cached tqdm-4.66.5-py3-none-any.whl.metadata (57 kB)
  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached httpcore-1.0.6-py3-none-any.whl.metadata (21 kB)
  Using cached h11-0.14.0-py3-none-any.whl.metadata (8.2 kB)
  Using cached annotated_types-0.7.0-py3-none-any.whl.metadata (15 kB)
  Using cached pydantic_core-2.23.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (6.6 kB)
Using cached openai-1.52.0-py3-none-any.whl (386 kB)
Using cached anyio-4.6.2.post1-py3-none-any.whl (90 kB)
Using cached 

In [2]:
from openai import OpenAI
client = OpenAI()

assistant = client.beta.assistants.create(
  name="Math Tutor",
  instructions="You are a personal math tutor. Write and run code to answer math questions.",
  tools=[{"type": "code_interpreter"}],
  model="gpt-4o",
)

In [3]:
thread = client.beta.threads.create()

In [4]:
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="I need to solve the equation `3x + 11 = 14`. Can you help me?"
)

In [6]:
print(thread.id, assistant.id)

thread_ZZ6CfXSEWNnkLv7rUMrDNiYE asst_b6i8cBLvOndN5QN2ObM7Z008


In [5]:
from typing_extensions import override
from openai import AssistantEventHandler
 
# First, we create a EventHandler class to define
# how we want to handle the events in the response stream.
 
class EventHandler(AssistantEventHandler):    
  @override
  def on_text_created(self, text) -> None:
    print(f"\nassistant > ", end="", flush=True)
      
  @override
  def on_text_delta(self, delta, snapshot):
    print(delta.value, end="", flush=True)
      
  def on_tool_call_created(self, tool_call):
    print(f"\nassistant > {tool_call.type}\n", flush=True)
  
  def on_tool_call_delta(self, delta, snapshot):
    if delta.type == 'code_interpreter':
      if delta.code_interpreter.input:
        print(delta.code_interpreter.input, end="", flush=True)
      if delta.code_interpreter.outputs:
        print(f"\n\noutput >", flush=True)
        for output in delta.code_interpreter.outputs:
          if output.type == "logs":
            print(f"\n{output.logs}", flush=True)
 
# Then, we use the `stream` SDK helper 
# with the `EventHandler` class to create the Run 
# and stream the response.
 
with client.beta.threads.runs.stream(
  thread_id=thread.id,
  assistant_id=assistant.id,
  instructions="Please address the user as Jane Doe. The user has a premium account.",
  event_handler=EventHandler(),
) as stream:
  stream.until_done()


assistant > Of course, Jane Doe! To solve the equation \(3x + 11 = 14\), we can follow these steps:

1. Subtract 11 from both sides to isolate the term with \(x\): 
   \[ 3x = 14 - 11 \]
2. Simplify the right side: 
   \[ 3x = 3 \]
3. Divide both sides by 3 to solve for \(x\):
   \[ x = \frac{3}{3} \]

Let's calculate this value:
assistant > code_interpreter

# Solve the equation 3x + 11 = 14
x = (14 - 11) / 3
x
assistant > The solution to the equation \(3x + 11 = 14\) is \(x = 1.0\).